In [1]:
import numpy as np
import pandas as pd

In [ ]:
import json
import requests
from bs4 import BeautifulSoup


In [ ]:

st_accept = "text/html"  # говорим веб-серверу,
# что хотим получить html
# имитируем подключение через браузер Mozilla на macOS
st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
# формируем хеш заголовков
headers = {
    "Accept": st_accept,
    "User-Agent": st_useragent
}


def get_links_from_map():
    # отправляем запрос с заголовками по нужному адресу
    req = requests.get("https://tabiturient.ru/map/", headers)
    # считываем текст HTML-документа
    src = req.text

    soup = BeautifulSoup(src, 'html.parser')
    # Извлекаем все теги <a> с атрибутом href
    links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
    with open('../../../data/links.txt', 'w') as f:
        for link in links:
            f.write(f"{link}\n")

In [2]:
data = pd.read_csv('../../../data/links.txt')

In [3]:
data.columns = ["url"]

In [4]:
data

,url
0,https://tabiturient.ru
1,https://tabiturient.ru/np
2,https://tabiturient.ru/calculator
3,https://tabiturient.ru/calculator
4,https://tabiturient.ru/calculator/pay
...,...
77316,https://tabiturient.ru/feedback
77317,https://tabiturient.ru/admin2/ca.php
77318,https://tabiturient.ru/adv
77319,https://tabiturient.ru/vuzam2


In [5]:
pattern = r"https://tabiturient\.ru/vuzu/[A-Za-z0-9_.-]+/proxodnoi$"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu = data[mask]

In [6]:
data_vizu

,url
41,https://tabiturient.ru/vuzu/agatu/proxodnoi
68,https://tabiturient.ru/vuzu/bigpibiysk/proxodnoi
102,https://tabiturient.ru/vuzu/altgaki/proxodnoi
134,https://tabiturient.ru/vuzu/agiki/proxodnoi
156,https://tabiturient.ru/vuzu/asmu/proxodnoi
...,...
25811,https://tabiturient.ru/vuzu/theatrinsyar/proxo...
25819,https://tabiturient.ru/vuzu/ystu/proxodnoi
25875,https://tabiturient.ru/vuzu/uniyarac/proxodnoi
25916,https://tabiturient.ru/vuzu/yarpgups/proxodnoi


In [7]:
data_vizu.to_csv('vizu_lins.csv', index=False)

In [8]:
pattern = r"https://tabiturient\.ru/vuzu/.*/proxodnoi$"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu_dup = data[mask]

In [9]:
data_vizu_dup

,url
41,https://tabiturient.ru/vuzu/agatu/proxodnoi
68,https://tabiturient.ru/vuzu/bigpibiysk/proxodnoi
102,https://tabiturient.ru/vuzu/altgaki/proxodnoi
134,https://tabiturient.ru/vuzu/agiki/proxodnoi
156,https://tabiturient.ru/vuzu/asmu/proxodnoi
...,...
25811,https://tabiturient.ru/vuzu/theatrinsyar/proxo...
25819,https://tabiturient.ru/vuzu/ystu/proxodnoi
25875,https://tabiturient.ru/vuzu/uniyarac/proxodnoi
25916,https://tabiturient.ru/vuzu/yarpgups/proxodnoi


In [10]:
exclusive_df1 = data_vizu_dup[~data_vizu_dup['url'].isin(data_vizu['url'])]

In [11]:
exclusive_df1

,url
452,https://tabiturient.ru/vuzu/ачгаа/proxodnoi
1929,https://tabiturient.ru/vuzu/билгу/proxodnoi
5049,https://tabiturient.ru/vuzu/дрти/proxodnoi


In [12]:
pattern = r"https://tabiturient\.ru/vuzu/[A-Za-z0-9_.-]+/obsh"

# Проверяем, содержит ли хотя бы одна ячейка строки нужный шаблон
mask = data.apply(lambda row: row.str.contains(pattern, regex=True, na=False).any(), axis=1)
data_vizu_obsh = data[mask]

In [13]:
data_vizu_obsh

,url
65,https://tabiturient.ru/vuzu/agatu/obsh
99,https://tabiturient.ru/vuzu/bigpibiysk/obsh
131,https://tabiturient.ru/vuzu/altgaki/obsh
153,https://tabiturient.ru/vuzu/agiki/obsh
164,https://tabiturient.ru/vuzu/asmu/obsh
...,...
25816,https://tabiturient.ru/vuzu/theatrinsyar/obsh
25872,https://tabiturient.ru/vuzu/ystu/obsh
25913,https://tabiturient.ru/vuzu/uniyarac/obsh
25923,https://tabiturient.ru/vuzu/yarpgups/obsh


```
{
	vuz: {
		long_name: string // полное название,
		short_name: string // аббревиатура вуза
		geolocation: string // регион
		is_goverment: boolean // True если гос вуз
		rating: string // рейтинг (пока буквами)
		logo: string // ссылка на фото
		website: string // ссылка на сайт вуза
	},
	programs: [
		{
			!direction: string // направление
			!profile: string // профиль
			!program_code: string // код программы
			!vuz: string // аббревиатура вуза
			!faculty: string // факультет
			!exams: [[string]] // набор экзаменов
			!scores: [[score, form]] // баллы
			!education_form: string // форма обучения
			?free_places: int // бюджетных мест
			?average_score: float // средни проходной балл
			?olympic: int // по олимпиаде поступило
			?price: int // стоимость обучения
		}
	]

}
```

In [9]:
line = {"hui": "ручка"}
json.dumps(line)

# for key value:
#     if key = exams
#       put_in_csv(json.dumps(value))

'{"hui": "\\u0440\\u0443\\u0447\\u043a\\u0430"}'

In [22]:
dict1 = {'forms': [
    {
        'education_form2': 'Очная',
        'score': 'Только платное',
        'free_places': 'Нет данных',
        'average_score': 'no data',
        'olympic': 'no data',
        'price': -319100
    },
    {
        'education_form2': 'Заочная',
        'score': 'Только платное',
        'free_places': 'Нет данных',
        'average_score': 'no data',
        'olympic': 'no data',
        'price': -80000
    }
]
}

In [23]:
dict1 = dict1['forms']

In [30]:

str1 = json.dumps(dict1, ensure_ascii=False)
str1

'[{"education_form2": "Очная", "score": "Только платное", "free_places": "Нет данных", "average_score": "no data", "olympic": "no data", "price": -319100}, {"education_form2": "Заочная", "score": "Только платное", "free_places": "Нет данных", "average_score": "no data", "olympic": "no data", "price": -80000}]'

In [31]:
list1 = [['РЯ'], ['M'], ['ИКТ', 'Ф']]
str1 =json.dumps(list1, ensure_ascii=False)
str1

'[["РЯ"], ["M"], ["ИКТ", "Ф"]]'

In [37]:
# Читаем исходный CSV-файл
df = pd.read_csv('../../../data/vizu_lins.csv')

# Разбиваем DataFrame на 4 части равного размера по числу строк
parts = np.array_split(df, 4)

# Сохраняем каждую часть в отдельный файл
for i, part in enumerate(parts, start=1):
    part.to_csv(f'data/vizu_lins_part{i}.csv', index=False)
    print(f'Часть {i} сохранена в файл vizu_lins_part{i}.csv')

Часть 1 сохранена в файл vizu_lins_part1.csv
Часть 2 сохранена в файл vizu_lins_part2.csv
Часть 3 сохранена в файл vizu_lins_part3.csv
Часть 4 сохранена в файл vizu_lins_part4.csv


/home/user/Desktop/PoVuzam/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
import  requests

st_accept = "text/html"  # говорим веб-серверу,
# что хотим получить html
# имитируем подключение через браузер Mozilla на macOS
# формируем хеш заголовков
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'uk,en-US;q=0.9,en;q=0.8,ru;q=0.7',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
}


def get_links_from_map():
    # отправляем запрос с заголовками по нужному адресу
    req = requests.get("https://ishim.utmn.ru/", headers)
    # считываем текст HTML-документа
    print(req)

In [9]:
get_links_from_map()

<Response [403]>


In [14]:
data_vizu_duplicates = data_vizu[data_vizu.duplicated(keep=False)]
data_vizu_duplicates


,url
6365,https://tabiturient.ru/vuzu/iptdnn/proxodnoi
6370,https://tabiturient.ru/vuzu/iptdnn/proxodnoi
